In [11]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re

%matplotlib inline

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 999)

In [3]:
directorio_actual = os.getcwd()
print("El directorio de trabajo actual es:", directorio_actual)


El directorio de trabajo actual es: g:\Unidades compartidas\Rocío doctorado\Proyecto NLP\Códigos


Leemos base de junio en formato csv y recreamos los tweets originales

In [4]:
datos_tweets_junio=pd.read_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/tweets_junio_procesado.csv',  encoding='utf8')

In [5]:
datos_tweets_junio_latin=pd.read_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/tweets_junio_procesado.csv',  encoding='latin1')

In [6]:
# # Filtrar tweets que no están cortados
datos_tweets_junio_latin = datos_tweets_junio_latin[~datos_tweets_junio_latin['full_text'].str.endswith('¦')]
# Obtener los índices de los tweets no cortados
indices_no_cortados = datos_tweets_junio_latin.index

# Filtrar otra tabla usando los índices de los tweets no cortados
datos_filtrados = datos_tweets_junio.loc[indices_no_cortados]


In [8]:
datos_filtrados.head(5)

,retweet_count,full_text,posicion,user_name,user_id
2,0,En el fondo lo que les jode es nuestra libertad #NoVotenContraLasMujeres #AbortoLegalYA,si,Luz Scarpati,2999619141
3,0,"holla! me gustaría q ayuden a esta junta d firmas para el aborto seguro y gratuito, gracias Aprueben la Ley de Aborto Seguro y Gratuito @DiputadosAR #abortolegalya #13JAbortoLegal https://t.co/aT9bKgrh9P via @ChangeorgAR",si,gigi not hadid,3668114236
6,0,Porque somos dueñas de nuestrxs cuerpas #NoVotenContraLasMujeres,si,Malena Bianca,66200186
9,0,"#NoVotenContraLasMujeres estoy tan a la mitad de tener un postura definida en este tema!!entiendo y comprendo a ambas partes,es muy triste e injusto para las dos partes!!",si,Cecilia,1005138863784976385
15,0,@MatiasRodrigTDF @LuzScarpati Que los @DiputadosAR #NoVotenContraLasMujeres #QueSeaLey,si,Elida Rechi,1475112482


In [9]:
datos_filtrados.to_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/tweets_junio_recortados.csv', index=False)

In [10]:
df_tweets_junio=datos_filtrados
df_tweets_junio.shape

(392964, 5)

Limpiamos los textos de los tweets

In [12]:
# Descargar las stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))  # Cambiar el idioma si es necesario

# Función para limpiar los tweets
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Eliminar URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Eliminar menciones de usuarios
    tweet = re.sub(r'#(\w+)', r'\1', tweet)  # Eliminar el '#' de los hashtags, dejando el texto
    #tweet = re.sub(r'#(\w+)', '', tweet)  # Eliminar el '#' de los hashtags, dejando el texto
    tweet = re.sub(r'[^\w\s]', '', tweet)  # Eliminar puntuación
    tweet = tweet.lower()  # Convertir a minúsculas
    tweet = re.sub(r'\brt\b', '', tweet, flags=re.IGNORECASE)  # Eliminar 'RT'
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])  # Eliminar stopwords
    return tweet

# Limpiar los tweets en la columna 'text'
df_tweets_junio['clean_text'] = df_tweets_junio['full_text'].apply(clean_tweet)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rocio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_tweets_junio['clean_text']

2                                                                                                 fondo jode libertad novotencontralasmujeres abortolegalya
3                holla gustaría q ayuden junta d firmas aborto seguro gratuito gracias aprueben ley aborto seguro gratuito abortolegalya 13jabortolegal via
6                                                                                                           dueñas nuestrxs cuerpas novotencontralasmujeres
9                                  novotencontralasmujeres tan mitad tener postura definida temaentiendo comprendo ambas parteses triste injusto dos partes
15                                                                                                                        novotencontralasmujeres quesealey
                                                                                 ...                                                                       
1041566                                                         

Observamos que las posiciones 'si' y 'no' están desbalanceadas

In [14]:
df_tweets_junio['posicion'].value_counts()

posicion
si    288761
no    104203
Name: count, dtype: int64

Realizamos un submuestreo de la clase mayoritaria

In [35]:
# Identificar la clase minoritaria y mayoritaria
conteo_clases = df_tweets_junio['posicion'].value_counts()
clase_minoritaria = conteo_clases.idxmin()  # Clase con menos instancias
clase_mayoritaria = conteo_clases.idxmax()  # Clase con más instancias

# Filtrar los DataFrames por clases
df_minoritaria = df_tweets_junio[df_tweets_junio['posicion'] == clase_minoritaria]
df_mayoritaria = df_tweets_junio[df_tweets_junio['posicion'] == clase_mayoritaria]

# Submuestreo de la clase mayoritaria
df_mayoritaria_submuestreada = df_mayoritaria.sample(len(df_minoritaria), random_state=42)

# Combinar los DataFrames
df_balanceado = pd.concat([df_mayoritaria_submuestreada, df_minoritaria])

# Barajar el DataFrame resultante
df_balanceado = df_balanceado.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar el balance final de las clases
print(df_balanceado['posicion'].value_counts())


posicion
no    104203
si    104203
Name: count, dtype: int64


Transformamos el texto en una representación numérica utilizando CountVectorizer

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

# Descargar stopwords en español
nltk.download('stopwords')
stop_words = list(stopwords.words('spanish'))  # Convertir el set a lista

# Vectorizar los tweets usando la lista de stopwords
vectorizer = CountVectorizer(max_df=0.99, min_df=10000, stop_words=stop_words)  # Usa la lista de stopwords
X = vectorizer.fit_transform(df_balanceado['clean_text'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rocio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
from sklearn.decomposition import LatentDirichletAllocation

# Configurar el número de tópicos (ajustar según el caso)
num_topics = 2

# Entrenar el modelo LDA
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)


LatentDirichletAllocation(n_components=2, random_state=42)

In [38]:
# Mostrar las palabras más importantes por tópico
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Tópico {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Mostrar los temas
no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
display_topics(lda, feature_names, no_top_words)


Tópico 1:
abortolegalya aborto noalaborto quesealey si abortosesionhistorica argentina favor mujeres abortolegal
Tópico 2:
argentinaesprovida salvemoslas2vidas vida noalabortoenargentina abortoseraley hoy abortolegal elijamoslas2vidas mujeres si


In [39]:
X_nuevo = vectorizer.transform(df_tweets_junio['clean_text'])

In [40]:
X_nuevo

<392964x17 sparse matrix of type '<class 'numpy.int64'>'
	with 636574 stored elements in Compressed Sparse Row format>

In [41]:
# Asignar el tópico más probable a cada tweet
topic_values = lda.transform(X_nuevo)
df_tweets_junio['topico'] = topic_values.argmax(axis=1)

# Ver los resultados
print(df_tweets_junio[['full_text', 'topico', 'posicion']].head())

                                                                                                                                                                                                                         full_text  \
2                                                                                                                                          En el fondo lo que les jode es nuestra libertad #NoVotenContraLasMujeres #AbortoLegalYA   
3   holla! me gustaría q ayuden a esta junta d firmas para el aborto seguro y gratuito, gracias   Aprueben la Ley de Aborto Seguro y Gratuito @DiputadosAR #abortolegalya #13JAbortoLegal https://t.co/aT9bKgrh9P via @ChangeorgAR   
6                                                                                                                                                                 Porque somos dueñas de nuestrxs cuerpas #NoVotenContraLasMujeres   
9                                                       #NoVotenContraLasMujeres

In [42]:
tabla_porcentaje = pd.crosstab(df_tweets_junio['topico'], df_tweets_junio['posicion'], normalize='columns') * 100 

# Mostrar la tabla de contingencia con porcentajes
print(tabla_porcentaje)


posicion         no         si
topico                        
0         19.946643  78.754056
1         80.053357  21.245944


In [47]:
df_tweets_junio.to_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/tweets_junio_LDA.csv')

## Selección de muestras para evaluación anecdótica

In [48]:
datos = pd.read_csv('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/tweets_junio_LDA.csv')

In [50]:
datos.columns

Index(['Unnamed: 0', 'retweet_count', 'full_text', 'posicion', 'user_name',
       'user_id', 'clean_text', 'topico'],
      dtype='object')

In [51]:
unicos_user_name=datos.drop_duplicates(['user_name'])
unicos_user_name['orden'] = range(len(unicos_user_name))
muestras = []

for nombre_topico, grupo in unicos_user_name.groupby('topico'):
    n_samples = min(40, len(grupo))
    
    if n_samples > 0:  # Asegurarse de que hay usuarios disponibles
        muestra = grupo.sample(n=n_samples, random_state=42, replace=False)
        muestras.append(muestra)

# Combinar todas las muestras en un DataFrame
muestra_topicos = pd.concat(muestras).reset_index(drop=True)

# Paso 4: Seleccionar solo las columnas 'user_name', 'full_text' y 'topico'
muestra_topicos = muestra_topicos[['user_name', 'full_text', 'topico']]

# Paso 5: Unir con el DataFrame original para recuperar el orden
muestra_topicos = muestra_topicos.merge(unicos_user_name[['user_name', 'orden']], on='user_name', how='left')

# Paso 6: Ordenar por la columna 'orden' y eliminar la columna 'orden'
muestra_topicos = muestra_topicos.sort_values(by='orden').drop(columns='orden')

# Paso 7: Agregar la columna 'Clasificacion' como vacía
muestra_topicos['Clasificacion'] = ''

# Paso 8: Reordenar las columnas para que 'comunidad' esté al principio
muestra_topicos = muestra_topicos[['topico', 'user_name', 'full_text', 'Clasificacion']]

# Supongamos que ya tienes tu DataFrame llamado muestra_comunidades
# muestra_topicos = muestra_topicos.sample(frac=1)

muestra_topicos

C:\Users\rocio\AppData\Local\Temp/ipykernel_6700/454498174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unicos_user_name['orden'] = range(len(unicos_user_name))


,topico,user_name,full_text,Clasificacion
16,0,paloma,"luchando juntas, siempre. ✊💚 #AbortoLegalYa https://t.co/MhelBZqQ0v",
49,1,Flor Wosh,"RT @CandelariaBotto: Las ricas abortan, las pobres mueren. #abortolegal es justicia social https://t.co/0iBKdZrIDl",
28,0,Marcela Godoy💚,"Hoy una periodista @Radio10 Paraná dijo estar en contra del aborto legal narrando: ""les introducen cosas peligrosas antinaturales"" . Una periodista. Comunicadora. Dicen barbaridades con total normalidad. Sus palabras describian un aborto clandestino. #AbortoLegalYa 💚",
22,0,fiminizi🐢,4 días para la votación en diputados. #abortolegalya,
41,1,Mujer❤Maravilla,"@Florfreijo Hay la quien elije la vida, obligan a su hija 16 años a tener su hijo ...ya le tenía casa para que se junto con su marido , hoy 21 años con 2 hijos y parece de 30 años , como le paso la vida tan rápido a esa criatura ... por eso #abortolegalya",
39,0,Iaru💜,"RT @delfi_benavidez: lo escribí yo, espero que no salgan a bardearme porque es simplemente mi opinión #AbortoLegalYA https://t.co/sNHpiLNing",
31,0,Cristina .,RT @MaviiVidela: 💚JUNTOS PODEMOS💚 #AbortoLegalYa #AbortoLegalSeguroYGratuito https://t.co/clHX8Vn1ao,
61,1,Jenifer Correa,Esas pendejas que están tomando él colegio por él #AbortoLegal cuantos años tiene ? 15? 16? 17? Xq no se ponen a estudiar y ser alguien en un futuro antes de andar haciendo marchas. #SiAlaVida #NoAlAborto,
21,0,"sabro,,",RT @paulitachaves: Pueden entender que no es ABORTO SI O ABORTO NO.... es LEGAL o CLANDESTINO... es facil... #AbortoLegalYa,
8,0,Ariel Sebas Borgna,"... el derecho es algo que va a permitir decidir sobre el propio destino de la vida. Gritamos, con la voz, con el cuerpo, con cada pañuelo verde: Educación Sexual para decidir Anticonceptivos para no abortar #AbortoLegalYa para no morir #QueSeaLey",


In [52]:
# Exportar el DataFrame a un archivo Excel con dos hojas
with pd.ExcelWriter('g:/Unidades compartidas/Rocío doctorado/Proyecto NLP/Datos/muestras_topicos.xlsx') as writer:
    muestra_topicos.to_excel(writer, sheet_name='Vanesa', index=False)
    muestra_topicos.to_excel(writer, sheet_name='Rocio', index=False)

# El archivo 'muestras_topicos.xlsx' tendrá dos hojas: 'Vanesa' y 'Rocio'